In [1]:
# Imports
import numpy as np
import pandas as pd

In [2]:
date = "2020-07-07"
fips = 51059
prop_beta = 1/3
file_name = f"{pd.Timestamp.today()}_sir-model-output.xlsx"

In [3]:
# Parameters
date = "2020-04-11"


In [4]:
# Read in usafacts data
case_df = pd.read_csv(
    "https://usafactsstatic.blob.core.windows.net/"
    "public/data/covid-19/covid_confirmed_usafacts.csv",
).set_index("countyFIPS").drop(
    # Remove unallocated cases (FIPS 0 or 1)
    range(2)
).reset_index().melt(
    # Melt dataframe (wide to long format)
    id_vars=["countyFIPS", "County Name", "State", "stateFIPS"],
    value_name="confirmed",
    var_name="date",
).astype({"date": "datetime64"})

In [5]:
# Create a day variable from the date variable
case_df = case_df.assign(
    days=(case_df["date"] - case_df["date"].min()).dt.days
)

# Trim up to but not including day 30
case_df = case_df[case_df["days"].ge(30)]

# Reverse order (highest to lowest day)
case_df = case_df.sort_values(["countyFIPS", "days"], ascending=False)

# Replace incorrect values with missing values
while case_df.groupby("countyFIPS")["confirmed"].pct_change().gt(0).any():
    case_df.loc[
        case_df.groupby("countyFIPS")["confirmed"].pct_change().gt(0),
        "confirmed",
    ] = np.nan

# Replace missing values with previous values
case_df.assign(
    confirmed=case_df["confirmed"].ffill()
)

# Restore the original order (lowest to highest day)
case_df = case_df.sort_values(["countyFIPS", "days"])

case_df = case_df.assign(
    # Calculate new cases from confirmed cases
    new_cases=case_df.groupby("countyFIPS")["confirmed"]
    .diff()
    .fillna(0)
)

In [6]:
# Read in population data
cens_df = pd.read_csv(
    "https://www2.census.gov/programs-surveys/popest/datasets/"
    "2010-2019/counties/totals/co-est2019-alldata.csv",
    usecols=[
        "STATE",
        "COUNTY",
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2019"
    ],
    encoding="latin-1"
)

# Combine state and county fips
cens_df = cens_df.assign(
    county_fips=(
        cens_df["STATE"].astype(str)
        + cens_df["COUNTY"].astype(str).str.zfill(3)
    ).astype(int)
)

In [7]:
# pop_df = pd.read_csv(
#     "https://usafactsstatic.blob.core.windows.net/"
#     "public/data/covid-19/covid_county_population_usafacts.csv",
#     ).set_index("countyFIPS").drop(
#         # Remove unallocated cases (FIPS 0)
#         0
#     ).reset_index()
# pop_df.head()

In [8]:
# Merge data files
df = case_df.merge(
    cens_df,
    left_on="countyFIPS",
    right_on="county_fips",
    how="left"
)

In [9]:
# Calculate model inputs
GAMMA = 1/7
# Growth rate
df = df.assign(
    gr=df.groupby("countyFIPS")["confirmed"].pct_change()
       / df.groupby("countyFIPS")["days"].diff()
)
# Calculate 7-day moving average of growth rate
df = df.assign(
    smooth_gr=df.groupby("countyFIPS")
    .rolling(window=7, min_periods=1)["gr"]
    .mean()
    .reset_index(level=0, drop=True)
)

# Calculate a cases added in previous 7 days - NW
df = df.assign(
    seven_day_new_cases = df.groupby("countyFIPS")["confirmed"]
    .diff(periods = 7)
)

# Doubling time and beta
df = df.assign(
    dt=np.log(2) / np.log(df["gr"] + 1),
    smooth_dt=np.log(2) / np.log(df["smooth_gr"] + 1),
    beta=(df["gr"] + GAMMA) / df["POPESTIMATE2019"],
    smooth_beta=(df["smooth_gr"] + GAMMA) / df["POPESTIMATE2019"],
)

# Rt
df = df.assign(
    rt=df["beta"] / GAMMA * df["POPESTIMATE2019"],
    smooth_rt=df["smooth_beta"] / GAMMA * df["POPESTIMATE2019"],
)

In [10]:
# Define sir function
from typing import Iterator, Tuple, Dict

import numpy as np

def sir(
        susceptible: int,
        total_infected: int,
        total_recovered: int,
        new_infected: int,
        new_recovered: int,
        beta: float,
        doubling_time: float,
        growth_rate: float,
        r_naught: float,
        gamma: float = 1 / 7,
        case_adjustment_factor: float = 10,
        start: int = 0,
        stop: int = 90
) -> Iterator[Tuple[
    int,
    float,
    float,
    float,
    float,
    float,
    float,
    float,
    float,
    float
]]:
    """
    Calculate the number of susceptible people and incidence and prevalence
    of infection and recovery in a population based on initial values.

    Core SIR model function that forecasts new
    - susceptible (S),
    - infected (I), and
    - recovered (R) values

    Arguments:
        susceptible: Initial number of susceptible people.
        total_infected: Initial number of total confirmed infections.
        total_recovered: Initial number of people who have recovered.
        new_infected: Initial number of new infections.
        new_recovered: Initial number of newly recovered individuals.
        beta: The effective contact rate during social distancing.
            BETA = (GROWTH_RATE + GAMMA) / SUSCEPTIBLE
        growth_rate: The rate of growth of total_cases.
            GROWTH_RATE = BETA * SUSCEPTIBLE - GAMMA
            GROWTH_RATE = 2 ** (1 / DOUBLING_TIME) - 1
        doubling_time: The time required for total_cases to double.
             DOUBLING_TIME = ln(2)/ln(GROWTH_RATE + 1)
        r_naught: The number of people a newly infected person will infect.
            R_NAUGHT = (GROWTH_RATE + GAMMA) / GAMMA
            R_NAUGHT = BETA / GAMMA * SUSCEPTIBLE
        gamma: The inverse of the recovery length, e.g.
            RECOVERY_TIME = 7
            GAMMA = 1 / RECOVERY_TIME
        case_adjustment_factor: A multiplier to estimate the true case count.
            Under-reporting of cases is common because many are asymptomatic.
            The case adjustment factor compensates for under-reporting.
        start: The day for which initial values are provided, e.g. day 0.
        stop: The total number of days to forecast (daily projections).

    Yields:
        Collected values across each day from start to end.
        1. day
        2. susceptible
        3. total infected (infected prevalence)
        4. total recovered (recovered prevalence)
        5. new cases (infected incidence)
        6. newly recovered individuals (recovered incidence)
        7. beta (recalculated)
        8. doubling time (recalculated)
        9. growth rate (recalculated)
        10. rt (recalculated)
    """
    # Yield initial values
    yield (
        start,
        susceptible,
        total_infected,
        total_recovered,
        new_infected,
        new_recovered,
        beta,
        doubling_time,
        growth_rate,
        r_naught
    )
    # Iterate over days and yield model outputs
    for day in range(start + 1, stop + 1):
        # New confirmed infections (new positive tests)
        infected_incidence = beta * total_infected * susceptible # This beta needs to be adjusted, March period and last 7 days
        # Number of infected who have just recovered
        recovered_incidence = gamma * total_infected
        # Number of people who are susceptible as of this time point
        susceptible_new = (
                susceptible
                - case_adjustment_factor
                * infected_incidence
        )
        # Total number of people who are infected as of this time point
        infected_prevalence = (
                infected_incidence
                + total_infected
                - recovered_incidence
        )
        # Total number of people who have recovered as of this time point
        recovered_prevalence = (
                recovered_incidence
                + total_recovered
                + infected_incidence * max(1., case_adjustment_factor - 1)
        )
        try:
            doubling_time_new = (
                    np.log(2) / np.log(infected_prevalence / total_infected)
            )
        except ZeroDivisionError:
            doubling_time_new = np.nan
        growth_rate_new = 2 ** (1 / doubling_time_new) - 1
        beta_new = (growth_rate_new + gamma) / susceptible_new
        rt = beta_new / gamma * susceptible_new
        yield (
            day,
            susceptible_new,
            infected_prevalence,
            recovered_prevalence,
            infected_incidence,
            recovered_incidence,
            beta_new,
            doubling_time_new,
            growth_rate_new,
            rt
        )
        susceptible, total_infected, total_recovered = (
            susceptible_new, infected_prevalence, recovered_prevalence
        )

In [11]:
# Get the pre-mitigation Beta estimate and More recent Beta estimate
# These two betas will be the min max used to be able to apply a fraction input to receive a new model beta
# such as 1/3, 2/3, 1/6
  
from datetime import datetime, timedelta
# Pre-mitigation
# Median doubling times in March
start = datetime.strptime("2020-03-01", "%Y-%m-%d")
end = datetime.strptime("2020-03-28", "%Y-%m-%d")

county_after_start = df["date"] >= start
county_before_end = df["date"] <= end
df = df.merge(
    df[
        county_after_start & county_before_end
        ].groupby("countyFIPS")["beta"].median().rename(
        "county_unmitigated_beta"
    ).reset_index(),
    on="countyFIPS",
    how="left"
)

# Recent history betas
recent_beta_end = df['date'].max()
recent_beta_start = recent_beta_end - timedelta(days = 7) 

county_recent_start = df["date"] >= recent_beta_start
county_recent_end = df["date"] <= recent_beta_end

df = df.merge(
    df[
        county_recent_start & county_recent_end
        ].groupby("countyFIPS")["beta"].median().rename(
        "county_mitigated_beta"
    ).reset_index(),
    on="countyFIPS",
    how="left"
)



In [12]:
from typing import Tuple, List
class MinMaxScaler:
    def __init__(self, values: Tuple) -> List:
        self.values = values
        self.min_val = min(values)
        self.max_val = max(values)
        self.max_min_diff = self.max_val - self.min_val

    def min_max_scale(self, x: Tuple):
        return [(x_i - self.min_val) / self.max_min_diff for x_i in x]

    def inverse_min_max_scale(self, x: Tuple):
        return [x_i * self.max_min_diff + self.min_val for x_i in x]

In [13]:
# Pick county and start date - Model Lever 1 and 2
county = df["countyFIPS"] == fips
start_date = df["date"] == pd.to_datetime(date)

# Get model inputs
subset = df[county & start_date]
s = subset["POPESTIMATE2019"].values[0]
#i = subset["confirmed"].values[0] this results in way to many infections to start with
i = subset["seven_day_new_cases"].values[0]

# Model Lever 3
#################### Use this Beta for a status quo model for now####################
b = subset["smooth_beta"].values[0]
#####################################################################################
# Get beta subset
# Scale to easily select a fractional thing between the min and max beta
betas = subset["county_unmitigated_beta"].values[0], subset["county_mitigated_beta"].values[0]
mms = MinMaxScaler(betas )
############################## Use this Beta for non status quo scenarios (ex 1/3 releaese day 0)#############
b = prop_beta * mms.max_min_diff + mms.min_val
##############################################################################################################

dt = subset["smooth_dt"].values[0]
gr = subset["smooth_gr"].values[0]
rt = subset["smooth_rt"].values[0]

In [14]:
# Get historical confirmed from day 0 onward
historical = df[county]
historical.to_excel("full history.xlsx")
historical = historical[historical["date"] >= pd.to_datetime(date)] #this date should be a variable instead of manual
historical = historical[["date", "new_cases"]].reset_index(drop=True)
historical['days'] = historical.index
#historical

In [15]:
# merge predictions and historical data
final = df.merge(historical, on=['days', 'date', 'new_cases'])
final.to_excel(file_name) # This should be automatically named

## Goal
There is a need to rapidy produce models for a variety of start dates and scenarios, both for OWS and Dr. Butler weekly modeling exercises. Currently, Advana does not have the capability to produce model results that do not adhere to the system already established to run on a regular interval, and for scenarios not yet programmed into Advana. The goal of this workbook is to support the Epi team with the ability to produce model results in an easy and automated way, both for analysis and for dissemination.

Functionaly, the Epis will need to be able to pull the following levers:
- The date the model starts
- A desired beta (i.e. 1/3 between full mitigation and no mitigation)
- The county for which the model is run

Currently these functions exist, but they need to be made more user friendly. Possible improvements to functionality include:
- Allow the user to specify the model start date once (currently requires entry twice)
- Allow the user to enter multiple sets of betas and run all models at once (currently only take one beta)
- Allow the user to enter multiple starting dates and run all models at once (currently only takes one starting date)
- Allow for the aggregation of the model results for each county requested. We commonly want to roll-up multiple counties that fall in the same catchment area for an MTF.
- Produce historical case counts for all counties requested (currently saves the full case history in one excel file and attaches case history that overlaps with the time period of the model in the same excel file as the model output. 
- Produces line charts with the model results, and idealy case count history for a length specified by the user
- Ability to run a model that is status quo on day zero, then alter the beta on a set date. Ex 1/3 Release Day 15

Athough this would be nice in dashboard form, speed in development is more important than pure ease-of-use.